# Speech emotion classification

In [5]:
import numpy as np
import pandas as pd
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import concatenate

Using TensorFlow backend.
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflo

# config parameters

In [6]:
# for demo, I selected only 4 features 
start_cols = 2
end_cols = 6

#model hyperparameters
timesteps = 2 # actually have 3 timesteps > 0,1,2
batch_size = 2
epochs = 100
neurons = 2

In [10]:
# create label by shifting 1 row
def create_label(df):
    df['id'] = df['_attribute_name_string'].astype(str).str[:2]
    df['label'] = df['emotion'].shift(-1)
    #shift time label
    df['label'] = df.groupby(['id'])['emotion'].shift(-1)
    return df

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    return df

# create timesteps
def get_sup(data):
    raw_values = data.values
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(raw_values, timesteps)
    supervised_values = supervised.values[timesteps:,:]
    return supervised_values 

# select features in dataframe
def select_features(df):
    lst = []
    for column in df.columns[start_cols:end_cols]:
        lst.append(get_sup(df[column]))
    return lst

#reformat into timesteps and remove nan, the more timestep, the more data is removed.
def re_format(a,b):
    a = np.array(a)    
    b = np.array(b)
    b = b.reshape(b.shape[1], b.shape[2], b.shape[0])
    
    def no_nan(list_a):
        return all(x == x for x in list_a) # oppsite of any(x!=x)

    filtered = [no_nan(x) for x in a]
    return np.array([y[-1] for y in a[filtered]]), b[filtered]

 # fit an LSTM network to training data
def fit_lstm(X, y, batch_size, nb_epoch, neurons):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    for i in range(nb_epoch):
        # if don't want to show training stages, set verbose=0
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

# validate data by average precision_score, can be changed depends on use cases
def predictor(model, batch_size, X, y):
    yhat = model.predict(X, batch_size=batch_size)
    average_precision = average_precision_score(y, yhat)
    return average_precision

In [11]:
def main():
    df = pd.read_csv('data_test.csv')
    # data preprocessing
    prep_df = create_label(df)
    y1 = get_sup(prep_df['label'])
    x1s = select_features(prep_df)
    y_all, x_all = re_format(y1, x1s)
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.3, random_state=42)
    # model fitting
    model = fit_lstm(X_train, y_train, batch_size, epochs, neurons)
    # forecasting
    average_precision = predictor(model, batch_size, X_test, y_test)
    print(f"average precision score: {average_precision}")

In [12]:
main()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
154/154 [==============================] - 0s 3ms/step - loss: 0.6173
Epoch 1/1
154/154 [==============================] - 0s 806us/step - loss: 0.4695
Epoch 1/1
154/154 [==============================] - 0s 908us/step - loss: 0.3709
Epoch 1/1
154/154 [==============================] - 0s 783us/step - loss: 0.3042
Epoch 1/1
154/154 [==============================] - 0s 776us/step - loss: 0.2493
Epoch 1/1
154/154 [==============================] - 0s 722us/step - loss: 0.1964
Epoch 1/1
154/154 [==============================] - 0s 730us/step - loss: 0.1568
Epoch 1/1
154/154 [==============================] - 0s 741us/step - loss: 0.1366
Epoch 1/1
154/154 [==============================] - 0s 773us/step - loss: 0.1257
Epoch 1/1
154/154 [==============================] - 0s 727us/step - loss: 0.1188
Epoch 1/1
154/154 [==============================] - 0s 727us/